{/* cspell:ignore Multibase, Quasicrystal */}

# QESEM: A Qiskit Function by Qedma

<LegacyContent>
<Admonition type="note" title="Notes">
* This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/qedma-qesem)
* Qiskit Functions are an experimental feature available only to IBM Quantum&reg; Premium Plan users. They are in preview release status and subject to change.
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note" title="Notes">
* This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/qedma-qesem)
* Qiskit Functions are an experimental feature available only to IBM Quantum&reg; Premium Plan users. They are in preview release status and subject to change.
</Admonition>
</CloudContent>

## Overview

While quantum processing units have vastly improved in recent years, errors due to noise and imperfections in existing hardware remain a central challenge for quantum algorithm developers. As the field approaches utility-scale quantum computations that cannot be verified classically, solutions for canceling noise with guaranteed accuracy are becoming increasingly important. To overcome this challenge, Qedma has developed Quantum Error Suppression and Error Mitigation (QESEM), seamlessly integrated on IBM Quantum Platform as a [Qiskit Function](/guides/functions).

With QESEM, users can run their quantum circuits on noisy QPUs to obtain highly accurate error-free results with highly efficient QPU-time overheads, close to fundamental bounds. To achieve this, QESEM leverages a suite of propriety methods developed by Qedma, for the characterization and reduction of errors. Error reduction techniques include gate optimization, noise-aware transpilation, error suppression (ES), and unbiased error mitigation (EM). With this combination of these characterization-based methods, users can achieve reliable, error-free results for generic large-volume quantum circuits, unlocking applications that cannot be accomplished otherwise.

## Description

You can use the QESEM function by Qedma to easily estimate and execute your circuits with error suppression and mitigation, achieving larger circuit volumes and higher accuracies. To use QESEM, you provide a quantum circuit, a set of observables to measure, a target statistical accuracy for each observable, and a chosen QPU. Before you run the circuit to the target accuracy, you can estimate the required QPU time based on an analytical calculation that does not require circuit execution. Once you are satisfied with the QPU time estimation, you can execute the circuit with QESEM.

When you execute a circuit, QESEM runs a device characterization protocol tailored to your circuit, yielding a reliable noise model for the errors occurring in the circuit. Based on the characterization, QESEM first implements noise-aware transpilation to map the input circuit onto a set of physical qubits and gates, which minimizes the noise affecting the target observable. These include the natively available gates (CX/CZ on IBM&reg; devices), as well as additional gates optimized by QESEM, forming QESEM’s extended gate set. QESEM then runs a set of characterization-based ES and EM circuits on the QPU and collects their measurement outcomes. These are then classically post-processed to provide an unbiased expectation value and an error bar for each observable, corresponding to the requested accuracy.

![Qedma QESEM overview](/images/guides/qedma-qesem/overview.svg)

QESEM has been demonstrated to provide high-accuracy results for a variety of quantum applications and on the largest circuit volumes achievable today. QESEM offers the following user-facing features, demonstrated in the benchmarks section below:
-	**Guaranteed accuracy:** QESEM outputs unbiased estimations for expectation values of observables. Its EM method is equipped with theoretical guarantees, which - together with Qedma’s cutting-edge characterization - ensure the mitigation converges to the noiseless circuit output up to the user-specified accuracy. In contrast to many heuristic EM methods that are prone to systematic errors or biases, QESEM’s guaranteed accuracy is essential for ensuring reliable results in generic quantum circuits and observables.
-	**Scalability to large QPUs:** QESEM’s QPU time depends on circuit volumes, but is otherwise independent of the number of qubits. Qedma has demonstrated QESEM on the largest quantum devices available today, including the IBM Quantum 127-qubit Eagle and 133-qubit Heron devices.
-	**Application-agnostic:** QESEM has been demonstrated on a variety of applications, including Hamiltonian simulation, VQE, QAOA, and amplitude estimation. Users can input any quantum circuit and observable to be measured, and obtain accurate error-free results. The only limitations are dictated by the hardware specifications and allocated QPU time, which determine the accessible circuit volumes and output accuracies. In contrast, many error reduction solutions are application-specific or involve uncontrolled heuristics, rendering them inapplicable for generic quantum circuits and applications.
-  **Extended gate set:** QESEM supports fractional-angle gates, and provides Qedma-optimized fractional-angle $Rzz(\theta)$ gates on IBM Quantum Eagle devices. This extended gate set enables more efficient compilation and unlocks circuit volumes larger by a factor of up to 2 compared to default CX/CZ compilation.
-	**Multibase observables:** QESEM supports input observables composed of many non-commuting Pauli strings, such as generic Hamiltonians. The choice of measurement bases and the optimization of QPU resource allocation (shots and circuits) is then performed automatically by QESEM to minimize the required QPU time for the requested accuracy. This optimization, which takes into account hardware fidelities and execution rates, enables you to run deeper circuits and obtain higher accuracies.

## Benchmarks

QESEM has been tested on a wide variety of use cases and applications. The following examples can assist you with assessing which types of workloads you can run with QESEM.

A key figure of merit for quantifying the hardness of both error mitigation and classical simulation for a given circuit and observable is **active volume**: the number of CNOT gates affecting the observable in the circuit. The active volume depends on the circuit depth and width, on the observable weight, and on the circuit structure, which determines the light cone of the observable. For further details, see the talk from the [2024 IBM Quantum Summit](https://www.youtube.com/watch?v=Hd-IGvuARfE&t=1730s). QESEM provides particularly large value in the high-volume regime, giving reliable results for generic circuits and observables.

![Active volume](/images/guides/qedma-qesem/active_volume.svg)


| Application    | Number of qubits | Device | Circuit description | Accuracy | Total time | Runtime usage |
| ---------  | ---------------- | ----- | -------------------------- | -------- | ---------- | ------------- |
| VQE circuit  | 8              | Eagle (r3) | 21 total layers, 9 measurement bases, 1D chain                    | 98%      | 35 min      | 14 min         |
| Kicked Ising   | 28              | Eagle (r3) | 3 unique layers x 3 steps, 2D heavy-hex topology                      | 97%     | 22 min      | 4 min          |
| Kicked Ising   | 28              | Eagle (r3) | 3 unique layers x 8 steps, 2D heavy-hex topology                     | 97%      | 116 min      | 23 min          |
| Trotterized Hamiltonian simulation   | 40  | Eagle (r3)            | 2 unique layers x 10 Trotter steps, 1D chain                    | 97%      | 3 hours     | 25 min         |
| Trotterized Hamiltonian simulation   | 119  | Eagle (r3)           | 3 unique layers x 9 Trotter steps, 2D heavy-hex topology                    | 95%      | 6.5 hours     | 45 min         |
| Kicked Ising  | 136             | Heron (r2) | 3 unique layers x 15 steps, 2D heavy-hex topology                    | 99%      | 52 min             | 9 min           |

Accuracy is measured here relative to the ideal value of the observable: $\frac{\langle O \rangle_{ideal} - \epsilon}{\langle O \rangle_{ideal}}$, where '$\epsilon$' is the absolute precision of the mitigation (set by the user input), and $\langle O \rangle_{ideal}$ is the observable at the noiseless circuit.
'Runtime usage' measures the usage of the benchmark in batch mode (sum over usage of individual jobs), whereas 'total time' measures usage in session mode (experiment wall time), which includes additional classical and communication times. QESEM is available for execution in both modes, so that users can make the best use of their available resources.

The 28-qubit Kicked Ising circuits simulate the Discrete Time Quasicrystal studied by Shinjo et al. (see [arXiv 2403.16718](https://arxiv.org/abs/2403.16718) and [Q2B24 Tokyo](https://www.youtube.com/watch?v=tQW6FdLc6zo)) on three connected loops of ibm_kawasaki. The circuit parameters taken here are $(\theta_x, \theta_z) = (0.9 \pi, 0)$, with a ferromagnetic initial state $| \psi_0 \rangle = | 0 \rangle ^{\otimes n}$. The measured observable is the absolute value of the magnetization $M = |\frac{1}{28} \sum_{i=0}^{27} \langle Z_i \rangle|$. The utility-scale Kicked Ising experiment was run on the 136 best qubits of ibm_fez; this particular benchmark was run at the Clifford angle $(\theta_x, \theta_z) = (\pi, 0)$, at which the active volume grows slowly with circuit depth, which - together with the high device fidelities - enables high accuracy at a short runtime.

Trotterized Hamiltonian simulation circuits are for a Transverse-Field Ising model at fractional angles: $(\theta_{zz}, \theta_x) = (\pi / 4, \pi /8)$ and $(\theta_{zz}, \theta_x) = (\pi / 6, \pi / 8)$ correspondingly (see [Q2B24 Tokyo](https://www.youtube.com/watch?v=tQW6FdLc6zo)). The utility-scale circuit was run on the 119 best qubits of ibm_brisbane, whereas the 40-qubit experiment was run on the best available chain. The accuracy is reported for the magnetization; high-accuracy results were obtained for higher-weight observables as well.

The VQE circuit was developed together with researchers from the Center for Quantum Technology and Applications at the Deutsches Elektronen-Synchrotron (DESY). The target observable here was a Hamiltonian consisting of a large number of non-commuting Pauli strings, emphasizing QESEM's optimized performance for multi-basis observables. Mitigation was applied to a classically-optimized ansatz; although these results are still unpublished, results of the same quality will be obtained for different circuits with similar structural properties.

## Get started
<LegacyContent>
Authenticate using your [IBM Quantum Platform API token](http://quantum.ibm.com/), and select the QESEM Qiskit Function as follows:
</LegacyContent>
<CloudContent>
Authenticate using your [IBM Quantum Platform API key](http://quantum.cloud.ibm.com/), and select the QESEM Qiskit Function as follows:
</CloudContent>

In [1]:
import qiskit

from qiskit_ibm_catalog import QiskitFunctionsCatalog

catalog = QiskitFunctionsCatalog()

qesem_function = catalog.load("qedma/qesem")

## Example

To get started, try this basic example of estimating the required QPU time to run QESEM for a given `pub`:

In [2]:
# This cell is hidden from users
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
instance = service.active_account()["instance"]
backend_name = service.least_busy().name

In [3]:
circ = qiskit.QuantumCircuit(5)
circ.cx(0, 1)
circ.cx(2, 3)
circ.cx(1, 2)
circ.cx(3, 4)

avg_magnetization = qiskit.quantum_info.SparsePauliOp.from_sparse_list(
    [("Z", [q], 1 / 5) for q in range(5)], num_qubits=5
)
other_observable = qiskit.quantum_info.SparsePauliOp.from_sparse_list(
    [("ZZ", [0, 1], 1.0), ("XZ", [1, 4], 0.5)], num_qubits=5
)

job = qesem_function.run(
    pubs=[(circ, [avg_magnetization, other_observable])],
    options={
        "estimate_time_only": "analytical",
    },
    instance=instance,
    backend_name=backend_name,  # E.g. "ibm_brisbane"
)

time_estimate_result = job.result()

The following example executes a QESEM job:

In [4]:
sample_job = qesem_function.run(
    pubs=[(circ, [avg_magnetization, other_observable])],
    instance=instance,
    backend_name=backend_name,  # E.g. "ibm_brisbane"
)

You can use the familiar Qiskit Serverless APIs to check your Qiskit Function workload's status or return results:

In [5]:
print(sample_job.status())
result = sample_job.result()

QUEUED


## Function parameters

<LegacyContent>
| Name |  Type | Description | Required | Default |  Example |
| -----| ------| ------------| -------- | ------- | -------- |
| `pubs` | [EstimatorPubLike](/guides/primitive-input-output) |This is the main input. The `Pub` contains 2-4 elements: a circuit, one or more observables, 0 or a single set of parameter values, and an optional precision. If a precision was not specified, then the `default_precision` from `options` will be used|  Yes|  N/A |  `[(circuit, [obs1,obs2,obs3], parameter_values, 0.03)]`  |
| `backend_name`| string|Name of the backend to use |No | QESEM will get least busy device reported by IBM| `"ibm_fez"`|
| `instance` | string|  The hub/group/project to use in that format |  No |  N/A | `“hub1/group1/project1”`  |
| `options` | dictionary |Input options. See **Options** section for more details. | No |  See the **Options** section for details.    |  `{ default_precision = 0.03, "max_execution_time" = 3600, "transpilation_level" = 0}`  |
</LegacyContent>
<CloudContent>
| Name |  Type | Description | Required | Default |  Example |
| -----| ------| ------------| -------- | ------- | -------- |
| `pubs` | [EstimatorPubLike](/guides/primitive-input-output) |This is the main input. The `Pub` contains 2-4 elements: a circuit, one or more observables, 0 or a single set of parameter values, and an optional precision. If a precision was not specified, then the `default_precision` from `options` will be used|  Yes|  N/A |  `[(circuit, [obs1,obs2,obs3], parameter_values, 0.03)]`  |
| `backend_name`| string|Name of the backend to use |No | QESEM will get least busy device reported by IBM| `"ibm_fez"`|
| `instance` | string|  The cloud resource name of the instance to use in that format |  No |  N/A | `“CRN”`  |
| `options` | dictionary |Input options. See **Options** section for more details. | No |  See the **Options** section for details.    |  `{ default_precision = 0.03, "max_execution_time" = 3600, "transpilation_level" = 0}`  |
</CloudContent>



### Options


| Option |  Choices | Description | Default |
| -----| -----------| -------- | ------- |
| `estimate_time_only` |  `"analytical"`  / `"empirical"` / None  | When set, the QESEM job will only calculate the QPU time estimation. Please see the following description for more details. When set to None, the circuit will be executed with QESEM| None |
|`default_precision` | 0 < float | Will apply to `pubs` that don't have presicion. The precision signifies the acceptable error on the expectation values of the observables in absolute value.  Namely, the QPU runtime for mitigation will be determined to provide output values for all the observables of interest that fall within a `1σ` confidence interval of the target precision. If multiple observables are provided, the mitigation will run until the target precision is reached for each of the input observables. | 0.02|
|`max_execution_time` | 0 < integer < 28,800 (8 hours)| Allows you to limit the QPU time, specified in seconds, to be used for the entire QESEM process. Please find additional details below.| 3,600 (one hour)|
| `transpilation_level` | 0 / 1 | See description below | 1|
| `execution_mode` | `"session"` / `"batch"` | See following description | "batch"|


<Admonition type="caution">The QPU time estimation changes from one backend to another. Therefore, when executing the QESEM function, make sure to run it on the same backend that was selected when obtaining the QPU time estimation. </Admonition>

<Admonition type="note">QESEM will end its run when it reaches the target precision or when it reaches `max_execution_time`, whichever comes first. </Admonition>

- `estimate_time_only` - This flag enables users to obtain an estimate for the QPU time required to execute the circuit with QESEM.
    - If set to `"analytical"`, an upper bound of the QPU time is calculated without consuming any QPU usage. This estimation has a 30-minute resolution (for example, 30 minutes, 60 minutes, 90 minutes, and so forth). It is typically pessimistic, and can only be obtained for single Pauli observables or sums of Paulis without intersecting supports (for example, Z0+Z1). It is primarily useful for comparing the complexity levels of different parameters provided by the user (circuit, accuracy, and so on).
    - To obtain a more accurate QPU time estimation, set this flag to `"empirical"`. Although this option requires running a small number of circuits, it provides a significantly more accurate QPU time estimation. This estimation has a 5-minute resolution (for example, 20 minutes, 25 minutes, 30 minutes, and so on). The user may choose to run the empirical time estimation in either batch or session mode. For more details, please see the `execution_mode` description. For example, in batch mode, empirical time estimation will consume less than 10 minutes of QPU time.

- `max_execution_time`: Allows you to limit the QPU time, specified in seconds, to be used for the entire QESEM process. Since the final QPU time required to reach the target accuracy is determined dynamically during the QESEM job, this parameter enables you to limit the cost of the experiment. If the dynamically-determined QPU time is shorter than the time allocated by the user, this parameter will not affect the experiment. The `max_execution_time` parameter is particularly useful in cases where the analytical time estimate provided by QESEM before the job starts is too pessimistic and the user wants to initiate a mitigation job anyway. After the time limit it reached, QESEM stops sending new circuits. Circuits that have already been sent continue running (so the total time may surpass the limit by up to 30 minutes), and the user receives the processed results from the circuits that ran up to that point. If you want to apply a QPU time limit shorter than the analytical time estimate, consult with Qedma to obtain an estimate for the accuracy achievable within the time limit.

- `transpilation_level`: After a circuit is submitted to QESEM, it automatically prepares several alternative circuit transpilations and chooses the one that minimizes QPU time. For instance, alternative transpilations might utilize Qedma-optimized fractional RZZ gates to reduce the circuit depth. Of course, all transpilations are equivalent to the input circuit, in terms of their ideal output. To exert more control over the circuit transpilation, set the transpilation level in the `options`. While `"transpilation_level": 1` corresponds to the default behavior described above, `"transpilation_level": 0` includes only minimal modifications required to the original circuit; for example, ‘layerification’ - the organization of circuit operations into ‘layers’ of simultaneous two-qubit gates. Note that automatic hardware-mapping onto high-fidelity qubits is applied in any case.

| transpilation_level | Description |
|:-:|:--|
| `1` | Default QESEM transpilation. Prepares several alternative transpilations and chooses the one that minimizes QPU time. Barriers may be modified in the layerification step. |
| `0` | Minimal transpilation: the mitigated circuit will closely resemble the input circuit structurally. Circuits provided in level 0 should match the device connectivity and should be specified in terms of the following gates: CX, Rzz(α), and standard single-qubit gates (U, x, sx, rz, and so on). Barriers will be respected in the layerification step. |

- `execution_mode` - The user can choose to run the QESEM job in either a dedicated [IBM session](/guides/execution-modes#session-mode) or across multiple [IBM batches](/guides/execution-modes#batch-mode):
    -   **Session Mode**: Sessions are more expensive but result in a shorter time-to-result. Once the session begins, the QPU is reserved exclusively for the QESEM job. The usage calculation includes both the time spent on QPU execution and the associated classical computations (performed by QESEM and IBM). The QESEM Qiskit Function takes care of creating and closing the session automatically. For users with unlimited access to QPUs (for example, on-premises setups), it is recommended to use session mode for faster QESEM execution.
    -   **Batch Mode**: In batch mode, the QPU is released during classical computations, leading to lower QPU usage. As batch jobs typically span a longer period, there is a greater risk of hardware drifts; QESEM incorporates measures to detect and compensate for drifts, retaining reliability over extended runs.



<Admonition type="note">
Barrier operations are typically used to specify the layers of two-qubit gates in quantum circuits. In level 0, QESEM preserves the layers specified by the barriers. In level 1, the layers specified by the barriers are considered as one transpilation alternative when minimizing QPU time.
</Admonition>

### Outputs
The output of a Circuit function is a [PrimitiveResult](/api/qiskit/qiskit.primitives.PrimitiveResult), which contains two fields:

- One [PubResult](/api/qiskit/qiskit.primitives.PubResult) object. It can be indexed directly from the `PrimitiveResult`.

- Job-level metadata.

Each `PubResult` contains a `data` and a `metadata` field.

- The `data` field contains at least an array of expectation values (`PubResult.data.evs`) and an array of standard errors (`PubResult.data.stds`). It can also contain more data, depending on the options used.

- The `metadata` field contains PUB-level metadata (`PubResult.metadata`).

The following code snippet describes how to retrieve the QPU time estimation (`estimate_time_only` is set):

In [6]:
print(
    f"The estimated QPU time for this PUB is: \n{time_estimate_result[0].metadata}"
)

The estimated QPU time for this PUB is: 
{'time_estimation_sec': 12600}


The following code snippet describes how to retrieve the mitigation results (`estimate_time_only` is not set).

In [7]:
print(f"The result of the submitted job had {len(result)} PUB")
print(
    f"The expectation values measured from this PUB are: \n{result[0].data.evs}"
)
print(f"The error-bar values are: \n{result[0].data.stds}")
print(
    f"And the associated metadata contains the original observables, making the results easier to interpret: \n{result[0].metadata}"
)

The result of the submitted job had 1 PUB
The expectation values measured from this PUB are: 
[1.00209222 0.98763303]
The error-bar values are: 
[0.00410043 0.01971415]
And the associated metadata contains the original observables, making the results easier to interpret: 
{'gate_fidelities': {'CNOT': 0.991932645993022, 'ID1Q': 0.9865222269129506}, 'total_shots': 445200, 'mitigation_shots': 180000, 'transpiled_circs': [{'circuit': 'OPENQASM 3.0;\ninclude "stdgates.inc";\nbit[16] c0;\nqubit[16] q1;\ncx q1[7], q1[6];\ncx q1[5], q1[4];\nbarrier q1[7], q1[6], q1[5], q1[4];\nbarrier q1[4], q1[5], q1[6], q1[7], q1[15];\ncx q1[6], q1[5];\ncx q1[4], q1[15];\nbarrier q1[6], q1[5], q1[4], q1[15];\nbarrier q1[4], q1[5], q1[6], q1[7], q1[15];\nbarrier q1[4], q1[5], q1[6], q1[7], q1[15];\nc0[4] = measure q1[4];\nc0[5] = measure q1[5];\nc0[6] = measure q1[6];\nc0[7] = measure q1[7];\nc0[15] = measure q1[15];\n', 'qubit_map': [[0, 7], [1, 6], [2, 5], [3, 4], [4, 15]], 'num_measurement_bases': 2}], 'no

## Fetch error messages

If your workload status is ERROR, use job.result() to fetch the error message to fetch the error message as follows:

In [8]:
print(job.result())

PrimitiveResult([PubResult(data=DataBin(), metadata={'time_estimation_sec': 12600})], metadata={})


## Get support

The Qedma support team is here to help! If you encounter any issues or have questions about using the QESEM Qiskit Function, please don't hesitate to reach out. Our knowledgeable and friendly support staff are ready to assist you with any technical concerns or inquiries you may have.

You can email us at support@qedma.com for assistance. Please include as much detail as possible about the issue you're experiencing to help us provide a swift and accurate response. You can also contact your dedicated Qedma POC representative via email or phone.

To help us assist you more efficiently, please provide the following information when you contact us:

- A detailed description of the issue
- The job ID
- Any relevant error messages or codes


We are committed to providing you with prompt and effective support to ensure you have the best possible experience with our Qiskit Function.

We are always looking to improve our product and we welcome your suggestions! If you have ideas on how we can enhance our services or features you'd like to see, please send us your thoughts at support@qedma.com

## Next steps

<Admonition type="tip" title="Recommendations">

- [Request access to Qedma QESEM](https://quantum.ibm.com/functions?id=7636f111-b7b8-4a00-b22d-5fcc5c5cb3d0)

</Admonition>